In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('database').getOrCreate()

24/11/21 19:28:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/21 19:28:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import (col,expr,count,countDistinct,datediff,to_date,date_add,year,month,lag,lead,rank,max,min,round,
                                   sum,when,lit,desc,coalesce,abs,greatest,least,array,array_sort,substring, explode,
                                   collect_list,array_intersect,datediff,unix_timestamp,rank,dense_rank,least,greatest,
                                   row_number
                                  )
from pyspark.sql.types import (StructField,StructType,
                    IntegerType,StringType,DateType,TimestampType )
from pyspark.sql import Window
from pyspark.sql import Row

#### 24 Customers Who Bought Products A and B but Not C MP

In [3]:
customers_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/24_customers.csv'))

In [4]:
orders_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/24_orders.csv'))

In [8]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)



In [10]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_name: string (nullable = true)



In [38]:
prdc_df = orders_df.filter(col('product_name') == lit('C')).select('customer_id')
prdc = [prdc_df.collect()[i][0] for i in range(0,len(prdc_df.collect()))]

orders_df.filter(col('product_name').isin(['A','B'])).select('customer_id')\
         .filter(~col('customer_id').isin(prdc)).groupBy('customer_id').agg(count((col('customer_id'))).alias('cust_count_id'))\
         .filter(col('cust_count_id') == 2).join(customers_df, orders_df.customer_id == customers_df.customer_id,'inner')\
         .select(customers_df.customer_id,customers_df.customer_name).show()

+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          3|    Elizabeth|
+-----------+-------------+



In [65]:
customers_df.createOrReplaceTempView('customersdf')
orders_df.createOrReplaceTempView('ordersdf')

spark.sql(' with c_id as (select customer_id from ordersdf where product_name == "C" ), \
               gpc_id as (select customer_id,count("customer_id") as cnt_id from ordersdf \
                          where product_name in ("A","B") and customer_id not in (select customer_id from c_id) group by customer_id) \
                          select gpc_id.customer_id,customer_name from gpc_id \
                          join customersdf on gpc_id.customer_id ==  customersdf.customer_id \
                          where gpc_id.cnt_id == 2' ).show()

+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          3|    Elizabeth|
+-----------+-------------+



#### 25 Highest Grade For Each Student MP

In [163]:
enrollments_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/25_enrollments.csv'))

In [164]:
enrollments_df.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- course_id: integer (nullable = true)
 |-- grade: integer (nullable = true)



In [166]:
mx_grade_df = enrollments_df.groupBy('student_id').agg(max(col('grade')).alias('mx_grade')) \
              .select(col('student_id').alias('stu_id'),col('mx_grade'))

lg_df = enrollments_df.join(mx_grade_df, (mx_grade_df.stu_id == enrollments_df.student_id) 
                      & (mx_grade_df.mx_grade == enrollments_df.grade),'inner' )\
                      .select('stu_id',enrollments_df.course_id,'mx_grade').orderBy('stu_id','course_id')

window_spec = Window.partitionBy(col('stu_id')).orderBy(col('stu_id'),col('course_id'))

lg_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1) \
     .select(col('stu_id').alias('student_id'),'course_id',col('mx_grade').alias('grade')).orderBy('student_id').show()

+----------+---------+-----+
|student_id|course_id|grade|
+----------+---------+-----+
|         1|        2|   99|
|         2|        2|   95|
|         3|        3|   82|
+----------+---------+-----+



In [204]:
enrollments_df.createOrReplaceTempView('enrollmentsdf')

spark.sql(''' 
    with mx_grade as (select student_id as stu_id ,max(grade) as mxgrade from enrollmentsdf group by student_id), 
               lg as (select student_id, course_id, mxgrade from enrollmentsdf join mx_grade on  
                      enrollmentsdf.student_id == mx_grade.stu_id where enrollmentsdf.grade == mx_grade.mxgrade),
              rnk as (select student_id,course_id,mxgrade,rank() over(partition by student_id 
                      order by course_id) as rnk from lg)
                     select student_id,course_id,mxgrade from rnk where rnk == 1 order by student_id 
       ''').show()

+----------+---------+-------+
|student_id|course_id|mxgrade|
+----------+---------+-------+
|         1|        2|     99|
|         2|        2|     95|
|         3|        3|     82|
+----------+---------+-------+



#### 26 Evaluate Boolean Expression MP

In [3]:
variables_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/26_variables.csv'))

In [4]:
expressions_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/26_expressions.csv'))

In [109]:
x = 66
y = 77
expressions_df.withColumn('xx',when(col('left_operand') == lit('x'),66) \
                              .when(col('left_operand') == lit('y'),77)) \
              .withColumn('yy',when(col('right_operand') == lit('x'),66) \
                              .when(col('right_operand') == lit('y'),77)) \
              .withColumn('value',when(col('operator') == '>', col('xx') > col('yy')) \
                                 .when(col('operator') == '<', col('xx') < col('yy')) \
                                 .when(col('operator') == '=', col('xx') == col('yy'))) \
                                 .select('left_operand','operator','right_operand','value').show()

+------------+--------+-------------+-----+
|left_operand|operator|right_operand|value|
+------------+--------+-------------+-----+
|           x|       >|            y|false|
|           x|       <|            y| true|
|           x|       =|            y|false|
|           y|       >|            x| true|
|           y|       <|            x|false|
|           x|       =|            x| true|
+------------+--------+-------------+-----+



In [110]:
expressions_df.createOrReplaceTempView('expressionsdf')

spark.sql(''' 
   with base as (select left_operand,operator,right_operand,case when left_operand == "x" then 66 else 77 end as xx,  
                        case when right_operand == "y" then 77 else 66 end as yy from expressionsdf) 
            select left_operand,operator,right_operand, case when operator = ">" then "xx" > "yy"
                                                            when operator = "<" then "xx" < "yy"
                                                            when operator = "=" then "xx" = "yy" end as value 
                                                             from base
  ''').show()

+------------+--------+-------------+-----+
|left_operand|operator|right_operand|value|
+------------+--------+-------------+-----+
|           x|       >|            y|false|
|           x|       <|            y| true|
|           x|       =|            y|false|
|           y|       >|            x|false|
|           y|       <|            x| true|
|           x|       =|            x|false|
+------------+--------+-------------+-----+



#### 27 Team Scores in Football Tournament MP

In [111]:
teams_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/27_teams.csv'))

In [112]:
matches_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/27_matches.csv'))

In [109]:
teams_df.printSchema()

root
 |-- team_id: integer (nullable = true)
 |-- team_name: string (nullable = true)



In [110]:
matches_df.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- host_team: integer (nullable = true)
 |-- guest_team: integer (nullable = true)
 |-- host_goals: integer (nullable = true)
 |-- guest_goals: integer (nullable = true)



In [42]:
one = matches_df.withColumnRenamed('host_team','team').withColumn('points',when( ( col('host_goals') - col('guest_goals') ) != -1,0)) \
                        .withColumn('num_points',when((col('host_goals') > col('guest_goals')),3)\
                        .when((col('host_goals') == col('guest_goals')),1)) \
                        .select('team','num_points')

In [45]:
two = matches_df.withColumnRenamed('guest_team','team').withColumn('points',when( ( col('host_goals') - col('guest_goals') ) != -1,0)) \
                        .withColumn('num_points',when((col('host_goals') < col('guest_goals')),3)\
                        .when((col('host_goals') == col('guest_goals')),1).otherwise(0)) \
                        .select('team','num_points')

In [61]:
one.union(two).groupBy(col('team')).agg(sum(col('num_points')).alias('num_points'))\
  .join(teams_df,one.team == teams_df.team_id,'right')\
  .select('team_id','team_name',coalesce(col('num_points'),lit('0')).alias('num_points')).show()

+-------+-----------+----------+
|team_id|  team_name|num_points|
+-------+-----------+----------+
|     10|Leetcode FC|         7|
|     20| NewYork FC|         3|
|     30| Atlanta FC|         1|
|     40| Chicago FC|         0|
|     50| Toronto FC|         3|
+-------+-----------+----------+



In [117]:
spark.sql('''
    with one as (select host_team as team, case when host_goals > guest_goals then 3 
                                             when host_goals == guest_goals then 1 end as num_points from matchesdf),
         two as (select guest_team as team,case when host_goals < guest_goals then 3 
                                              when host_goals == guest_goals then 1 end as num_points from matchesdf),
       three as (select team, sum(num_points) as num_points from (select * from one union all select * from two rigt) 
                                        group by team)
                 select team_id,team_name,coalesce(num_points,'0') as num_points from teamsdf left join three  on team_id == team
         ''').show()

+-------+-----------+----------+
|team_id|  team_name|num_points|
+-------+-----------+----------+
|     10|Leetcode FC|         7|
|     20| NewYork FC|         3|
|     30| Atlanta FC|         1|
|     40| Chicago FC|         0|
|     50| Toronto FC|         3|
+-------+-----------+----------+



#### 28 Apples & Oranges MP

In [141]:
sales_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/28_sales.csv'))

In [119]:
sales_df.printSchema()

root
 |-- sale_date: string (nullable = true)
 |-- fruit: string (nullable = true)
 |-- sold_num: integer (nullable = true)



In [120]:
sales_df.show()

+----------+-------+--------+
| sale_date|  fruit|sold_num|
+----------+-------+--------+
|2020-05-01| apples|      10|
|2020-05-01|oranges|       8|
|2020-05-02| apples|      15|
|2020-05-02|oranges|      15|
|2020-05-03| apples|      20|
|2020-05-03|oranges|       0|
|2020-05-04| apples|      15|
|2020-05-04|oranges|      16|
+----------+-------+--------+



In [136]:
apples_df = sales_df.filter(col('fruit') == 'apples')
oranges_df = sales_df.filter(col('fruit') == 'oranges').withColumnRenamed('sale_date','saledate').withColumnRenamed('sold_num','soldnum')
apples_df.join(oranges_df,apples_df.sale_date == oranges_df.saledate,'inner')\
.select('sale_date',expr("sold_num -soldnum ").alias('diiff')).show()

+----------+-----+
| sale_date|diiff|
+----------+-----+
|2020-05-01|    2|
|2020-05-02|    0|
|2020-05-03|   20|
|2020-05-04|   -1|
+----------+-----+



In [140]:
sales_df.createOrReplaceTempView('salesdf')

spark.sql('''
   with apple as (select sale_date,fruit,sold_num from salesdf where fruit = "apples"), 
      oranges as (select sale_date,fruit,sold_num from salesdf where fruit = "oranges") 
                  select apple.sale_date,apple.sold_num - oranges.sold_num  as diff from apple 
                         join oranges on apple.sale_date = oranges.sale_date
         ''').show()

+----------+----+
| sale_date|diff|
+----------+----+
|2020-05-01|   2|
|2020-05-02|   0|
|2020-05-03|  20|
|2020-05-04|  -1|
+----------+----+



#### 29 Number of Calls Between Two Persons PM

In [143]:
calls_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/29_calls.csv'))

In [144]:
calls_df.printSchema()

root
 |-- from_id: integer (nullable = true)
 |-- to_id: integer (nullable = true)
 |-- duration: integer (nullable = true)



In [174]:
calls_df.withColumn('pair_key',expr("array_sort(array(from_id,to_id))"))\
         .groupBy(col('pair_key')).agg(count(col('duration')).alias('call_count'),sum(col('duration')).alias('call_duration'))\
         .select( col('pair_key')[0].alias('person_01'),col('pair_key')[1].alias('person_02'),'call_count','call_duration').show()

+---------+---------+----------+-------------+
|person_01|person_02|call_count|call_duration|
+---------+---------+----------+-------------+
|        1|        2|         2|           70|
|        1|        3|         1|           20|
|        3|        4|         4|          999|
+---------+---------+----------+-------------+



In [180]:
calls_df.createOrReplaceTempView('callsdf')

spark.sql('''
            select a.pair_key[0] as from_id,a.pair_key[1] as to_id,call_count,duration from 
            (select array_sort(array(from_id,to_id))as pair_key,count(duration) as call_count,sum(duration) as duration 
                          from callsdf group by pair_key) a
         ''').show()

+-------+-----+----------+--------+
|from_id|to_id|call_count|duration|
+-------+-----+----------+--------+
|      1|    2|         2|      70|
|      1|    3|         1|      20|
|      3|    4|         4|     999|
+-------+-----+----------+--------+



#### 30 Countries You Can Safely Invest In MP

In [3]:
person_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/30_person.csv'))

In [4]:
country_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/30_country.csv'))

In [5]:
calls_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/30_calls.csv'))

In [184]:
person_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- phone_number: string (nullable = true)



In [185]:
country_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- country_code: integer (nullable = true)



In [186]:
calls_df.printSchema()

root
 |-- caller_id: integer (nullable = true)
 |-- callee_id: integer (nullable = true)
 |-- duration: integer (nullable = true)



In [262]:
total_df =calls_df.withColumn('callarrays',array_sort(array(col('caller_id'),col('callee_id')))) \
  .agg((sum(col('duration'))/count(col('duration'))).alias('total_avg'))

In [213]:
final_df= calls_df.withColumn('callarrays',array_sort(array(col('caller_id'),col('callee_id')))).groupBy(col('callarrays'))\
  .agg(count(col('duration')).alias('cnt'),sum(col('duration')).alias('sumduration'))

In [278]:
israel_df = final_df.withColumn("exploded_callarrays", explode(final_df["callarrays"])).filter(col('exploded_callarrays').isin([7,9]))\
.select( (sum(col('sumduration'))/sum(col('cnt'))).alias('israel')).select(lit('israel').alias('country'),col('israel').alias('avg'))


In [279]:
Morocco_df = final_df.withColumn("exploded_callarrays", explode(final_df["callarrays"])).filter(col('exploded_callarrays').isin([1,2]))\
.select( (sum(col('sumduration'))/sum(col('cnt'))).alias('Morocco')).select(lit('Morocco').alias('country'),col('Morocco').alias('avg'))


In [281]:
peru_df = final_df.withColumn("exploded_callarrays", explode(final_df["callarrays"])).filter(col('exploded_callarrays').isin([3,12]))\
.select( (sum(col('sumduration'))/sum(col('cnt'))).alias('Peru')).select(lit('Peru').alias('country'),col('Peru').alias('avg'))

In [294]:
israel_df.union(Morocco_df).union(peru_df).orderBy(desc('avg')).first()[0]

'Peru'

In [174]:
person_df.createOrReplaceTempView('persondf')
country_df.createOrReplaceTempView('countrydf')
calls_df.createOrReplaceTempView('callsdf')

spark.sql('''
  with code_list as (select collect_list(id) as coldelst,countrydf.name from person_df join countrydf on 
                       substring(person_df.phone_number,1,3) == countrydf.country_code group by countrydf.name),
      code_array as (select array_sort(array(caller_id,callee_id)) as code, count(duration) as count_call, 
                       sum(duration) as total_duration from callsdf group by code),
    explode_code as (select code,count_call,total_duration,explode(code) as ex_code from code_array),
           logic as (select country,sum(avg)/count(country) as avg from (select case when ex_code in (3,12) then "Peru"
                                 when ex_code in (1,2)  then "Morocco"
                                 when ex_code in (7,9)  then "Israel" end as country,
                            case when ex_code in (3,12) then round(sum(total_duration)/sum(count_call),2) 
                                 when ex_code in (1,2) then round(sum(total_duration)/sum(count_call),2) 
                                 when ex_code in (7,9) then round(sum(total_duration)/sum(count_call),2) end as avg 
                            from explode_code  group by ex_code) group by country),
      global_avg as (select sum(duration)/count(duration) as global_avg from callsdf )
                    select * from logic where avg > (select global_avg from  global_avg)
         ''').show()


+-------+------+
|country|   avg|
+-------+------+
|   Peru|145.67|
+-------+------+



#### 31 Tree Node M 

In [5]:
tree_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/31_tree.csv'))

In [176]:
tree_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- p_id: integer (nullable = true)



In [291]:
root_df = tree_df.withColumn('type',when((col('p_id')).isNull(),lit('root'))).filter(col('type') == lit('root'))\
.select('id','type')

leaf_df = tree_df.alias('d1').join(tree_df.alias('d2'),col('d1.id')==col('d2.p_id'),'anti')\
.withColumn('type',lit('leaf')).select('id','type')

inner_df = tree_df.alias('d1').join(tree_df.alias('d2'),col('d1.id')==col('d2.p_id'),'leftsemi')\
.withColumn('type',lit('inner')).select('id','type')\
.filter((col('id') != tree_df.filter(col('p_id').isNull()).select('id').collect()[0][0]))

root_df.union(leaf_df).union(inner_df).show()

In [300]:
   tree_df.withColumn('type',when((col('p_id')).isNull(),lit('root'))).filter(col('type') == lit('root'))\
   .select('id','type')\
.union(tree_df.alias('d1').join(tree_df.alias('d2'),col('d1.id')==col('d2.p_id'),'anti')\
   .withColumn('type',lit('leaf')).select('id','type'))\
.union(tree_df.alias('d1').join(tree_df.alias('d2'),col('d1.id')==col('d2.p_id'),'leftsemi')\
   .withColumn('type',lit('inner')).select('id','type')\
   .filter((col('id') != tree_df.filter(col('p_id').isNull()).select('id').collect()[0][0]))).orderBy('id').show()

+---+-----+
| id| type|
+---+-----+
|  1| root|
|  2|inner|
|  3| leaf|
|  4| leaf|
|  5| leaf|
+---+-----+



In [317]:
tree_df.createOrReplaceTempView('treedf')

spark.sql('''
  with root as (select id,'root' as type from treedf where p_id is null),
       leaf as (select id, 'leaf' as type from treedf as d1 anti join treedf as d2 on d1.id == d2.p_id ),
      inner as (select id, 'inner' as type from treedf as d1 left semi join treedf as d2 on d1.id == d2.p_id 
                    where id not in (select id from root))
                select * from root union(select * from leaf) union (select * from inner) order by id
         ''').show()

+---+-----+
| id| type|
+---+-----+
|  1| root|
|  2|inner|
|  3| leaf|
|  4| leaf|
|  5| leaf|
+---+-----+



#### 32 Game Play Analysis III

In [318]:
activity_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/32_activity.csv'))

In [319]:
activity_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- event_date: string (nullable = true)
 |-- games_played: integer (nullable = true)



In [357]:
#  to calculate a rolling sum over the previous 2 rows including the current row:
window_spec = Window.partitionBy('player_id').orderBy('player_id','event_date').rowsBetween(-2,0)

activity_df.withColumn('gameplayed',sum('games_played').over(window_spec))\
    .select('player_id','event_date',col('gameplayed')).show()
                                                                                 

+---------+----------+----------+
|player_id|event_date|gameplayed|
+---------+----------+----------+
|        1|2016-03-01|         5|
|        1|2016-05-02|        11|
|        1|2017-06-25|        12|
|        3|2016-03-02|         0|
|        3|2018-07-03|         5|
+---------+----------+----------+



In [373]:
activity_df.createOrReplaceTempView('activitydf')

spark.sql('''
          select player_id,event_date, sum(games_played) over(partition by player_id order by player_id,event_date
          ROWS BETWEEN  2 PRECEDING AND CURRENT ROW) as gameplayed from activitydf
         ''').show()

+---------+----------+----------+
|player_id|event_date|gameplayed|
+---------+----------+----------+
|        1|2016-03-01|         5|
|        1|2016-05-02|        11|
|        1|2017-06-25|        12|
|        3|2016-03-02|         0|
|        3|2018-07-03|         5|
+---------+----------+----------+



#### 33 Grand Slam Titles MP

In [5]:
players_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/33_players.csv'))

In [6]:
championships_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/33_championships.csv'))

In [376]:
players_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- player_name: string (nullable = true)



In [377]:
championships_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- Wimbledon: integer (nullable = true)
 |-- Fr_open: integer (nullable = true)
 |-- US_open: integer (nullable = true)
 |-- Au_open: integer (nullable = true)



In [378]:
championships_df.show()

+----+---------+-------+-------+-------+
|year|Wimbledon|Fr_open|US_open|Au_open|
+----+---------+-------+-------+-------+
|2018|        1|      1|      1|      1|
|2019|        1|      1|      2|      2|
|2020|        2|      1|      2|      2|
+----+---------+-------+-------+-------+



In [397]:
championships_df.select('year',col('Wimbledon').alias('title'))\
.union(championships_df.select('year',col('Fr_open').alias('title'))) \
.union(championships_df.select('year',col('US_open').alias('title'))) \
.union(championships_df.select('year',col('Au_open').alias('title')))\
.groupBy('title').agg(count(col('title')).alias('grand_slams_count')) \
.join(players_df,col('title') == col('player_id'),'inner' )\
.select('player_id','player_name','grand_slams_count').show()

+---------+-----------+-----------------+
|player_id|player_name|grand_slams_count|
+---------+-----------+-----------------+
|        1|      Nadal|                7|
|        2|    Federer|                5|
+---------+-----------+-----------------+



In [406]:
players_df.createOrReplaceTempView('playersdf')
championships_df.createOrReplaceTempView('championshipsdf')

spark.sql(''' 
  with main as (select title,count(title) as grand_slams_count from
                 (select Wimbledon as title from championshipsdf
                      union all(select Fr_open as title from championshipsdf)
                      union all(select US_open as title from championshipsdf)
                      union all(select Au_open as title from championshipsdf)) group by title)
                select player_id,player_name,grand_slams_count from playersdf join main on player_id == title

         ''').show()

+---------+-----------+-----------------+
|player_id|player_name|grand_slams_count|
+---------+-----------+-----------------+
|        1|      Nadal|                7|
|        2|    Federer|                5|
+---------+-----------+-----------------+



#### 34 Leetflex Banned Accounts MP

In [423]:
loginfo_schema = StructType([
                        StructField('account_id',IntegerType()),
                        StructField('ip_address',IntegerType()),
                        StructField('login',TimestampType()),
                        StructField('logout',TimestampType()),
                         ])

In [502]:
loginfo_df = (spark.read
               .option('header',True)
               .schema(loginfo_schema )
               .format('csv')
               .load('../../data/advsql/34_loginfo.csv')
               .withColumn('diff',unix_timestamp(col('logout')) - unix_timestamp(col('login')))  
               .withColumn('minutes',round( (col('diff')/60),2))
               .withColumn('hours',round( (col('diff')/3600),2)))

In [503]:
loginfo_df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- ip_address: integer (nullable = true)
 |-- login: timestamp (nullable = true)
 |-- logout: timestamp (nullable = true)
 |-- diff: long (nullable = true)
 |-- minutes: double (nullable = true)
 |-- hours: double (nullable = true)



In [504]:
loginfo_df.show()

+----------+----------+-------------------+-------------------+-----+-------+-----+
|account_id|ip_address|              login|             logout| diff|minutes|hours|
+----------+----------+-------------------+-------------------+-----+-------+-----+
|         1|         1|2021-02-01 09:00:00|2021-02-01 09:30:00| 1800|   30.0|  0.5|
|         1|         2|2021-02-01 08:00:00|2021-02-01 11:30:00|12600|  210.0|  3.5|
|         2|         6|2021-02-01 20:30:00|2021-02-01 22:00:00| 5400|   90.0|  1.5|
|         2|         7|2021-02-02 20:30:00|2021-02-02 22:00:00| 5400|   90.0|  1.5|
|         3|         9|2021-02-01 16:00:00|2021-02-01 16:59:59| 3599|  59.98|  1.0|
|         3|        13|2021-02-01 17:00:00|2021-02-01 17:59:59| 3599|  59.98|  1.0|
|         4|        10|2021-02-01 16:00:00|2021-02-01 17:00:00| 3600|   60.0|  1.0|
|         4|        11|2021-02-01 17:00:00|2021-02-01 17:59:59| 3599|  59.98|  1.0|
+----------+----------+-------------------+-------------------+-----+-------

In [508]:
loginfo_df.alias('d1').join(loginfo_df.alias('d2'),(col('d1.account_id') == col('d2.account_id') )
                            & (unix_timestamp(col('d1.login')) < unix_timestamp(col('d2.logout')) )
                            & (unix_timestamp(col('d1.logout')) > unix_timestamp(col('d2.login')) )
                            & (col('d1.ip_address') != col('d2.ip_address')), 'inner')\
          .select(col('d1.account_id')).show()

+----------+
|account_id|
+----------+
|         1|
|         1|
+----------+



#### 35 The Most Recent Three Orders MP

In [509]:
customers_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/35_customers.csv'))

In [510]:
orders_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/35_orders.csv'))

In [511]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [512]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- cost: integer (nullable = true)



In [533]:
window_spec = Window.partitionBy('name').orderBy(col('name').asc(),col('order_date').desc())
orders_df.join(customers_df,orders_df.customer_id == customers_df.customer_id,'inner')\
.select('name',orders_df.customer_id,'order_id','order_date' )\
.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') != 4)\
.select('name','customer_id','order_id','order_date').orderBy(col('name').asc(),col('order_date').desc()).show()

+---------+-----------+--------+----------+
|     name|customer_id|order_id|order_date|
+---------+-----------+--------+----------+
|Annabelle|          3|       7|2020-08-01|
|Annabelle|          3|       3|2020-07-31|
| Jonathan|          2|       9|2020-08-07|
| Jonathan|          2|       6|2020-08-01|
| Jonathan|          2|       2|2020-07-30|
|   Marwan|          4|       4|2020-07-29|
|  Winston|          1|       8|2020-08-03|
|  Winston|          1|       1|2020-07-31|
|  Winston|          1|      10|2020-07-15|
+---------+-----------+--------+----------+



In [549]:
customers_df.createOrReplaceTempView('customersdf')
orders_df.createOrReplaceTempView('ordersdf')

spark.sql('''
   with main as (select name,customersdf.customer_id,order_id,order_date from customersdf 
                    join ordersdf on customersdf.customer_id == ordersdf.customer_id),
         rnk as (select name,customer_id,order_id,order_date,rank() over(PARTITION BY name 
                    ORDER BY name ASC,order_date DESC) as rnk from main)
                select name,customer_id,order_id,order_date from rnk where rnk != 4 order by name asc, order_date desc
                
         ''').show()

+---------+-----------+--------+----------+
|     name|customer_id|order_id|order_date|
+---------+-----------+--------+----------+
|Annabelle|          3|       7|2020-08-01|
|Annabelle|          3|       3|2020-07-31|
| Jonathan|          2|       9|2020-08-07|
| Jonathan|          2|       6|2020-08-01|
| Jonathan|          2|       2|2020-07-30|
|   Marwan|          4|       4|2020-07-29|
|  Winston|          1|       8|2020-08-03|
|  Winston|          1|       1|2020-07-31|
|  Winston|          1|      10|2020-07-15|
+---------+-----------+--------+----------+



#### 36 Maximum Transaction Each Day MP

In [34]:
transactions_schema = StructType([
                        StructField('transaction_id',IntegerType()),
                        StructField('day',TimestampType()),
                        StructField('amount',IntegerType())
                         ])

In [45]:
transactions_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/36_transactions.csv')
               .withColumn('date',to_date('day','yyyy-M-dd HH:mm:ss')))

In [46]:
transactions_df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- date: date (nullable = true)



In [53]:
widonw_spec = Window.partitionBy('date').orderBy(col('amount').desc())

transactions_df.withColumn('rnk',dense_rank().over(widonw_spec)).filter(col('rnk') == 1)\
 .select('transaction_id').orderBy('transaction_id').show()

+--------------+
|transaction_id|
+--------------+
|             1|
|             5|
|             6|
|             8|
+--------------+



In [59]:
transactions_df.createOrReplaceTempView('transactionsdf')

spark.sql('''
          select transaction_id from 
          (select transaction_id,amount,date,dense_rank() OVER(PARTITION BY date ORDER BY amount DESC) as rnk
              from transactionsdf) where rnk == 1 order by transaction_id
          ''').show()

+--------------+
|transaction_id|
+--------------+
|             1|
|             5|
|             6|
|             8|
+--------------+



#### 37 Project Employees III MP

In [61]:
project_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/37_project.csv'))

In [62]:
employee_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/37_employee.csv'))

In [63]:
project_df.printSchema()

root
 |-- project_id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)



In [64]:
employee_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- experience_years: integer (nullable = true)



In [82]:
projectdf = project_df.join(employee_df,project_df.employee_id == employee_df.employee_id, 'inner' )\
.select('project_id',project_df.employee_id,'experience_years')

window_spec = Window.partitionBy('project_id').orderBy(col('experience_years').desc())

projectdf.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1)\
.select('project_id','employee_id').show()

+----------+-----------+
|project_id|employee_id|
+----------+-----------+
|         1|          1|
|         1|          3|
|         2|          1|
+----------+-----------+



In [90]:
project_df.createOrReplaceTempView('projectdf')
employee_df.createOrReplaceTempView('employeedf')

spark.sql('''
  with main as (select project_id,projectdf.employee_id as employee_id,experience_years from projectdf join employeedf on
                    projectdf.employee_id == employeedf.employee_id)
                select project_id,employee_id from  
                   (select project_id,employee_id,experience_years,rank() OVER(PARTITION BY project_id 
                       ORDER BY experience_years DESC) as rnk from main) where rnk = 1
         ''').show()

+----------+-----------+
|project_id|employee_id|
+----------+-----------+
|         1|          1|
|         1|          3|
|         2|          1|
+----------+-----------+



#### 38 Find the Start and End Number of Continuous Ranges MP

In [115]:
logs_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/38_logs.csv'))

In [98]:
logs_df.printSchema()

root
 |-- log_id: integer (nullable = true)



In [ ]:
''' 
df_grouped = df_with_groups.groupBy("group_id").agg(
    col("log_id").alias("end_id"),
    lag("log_id").over(Window.partitionBy("group_id").orderBy("log_id")).alias("start_id")
)
'''

In [213]:
window_spec = Window.orderBy("log_id")

df_with_lag = logs_df.withColumn("prev_log_id", lag("log_id").over(window_spec))

df_with_breaks = df_with_lag.withColumn("is_new_group", when((col("log_id") - col("prev_log_id")) > 1, 1).otherwise(0))

df_with_groups = df_with_breaks.withColumn("group_id", sum("is_new_group").over(Window.orderBy("log_id")))

logic_df = df_with_groups.withColumn('strend',rank().over(Window.partitionBy('group_id').orderBy('log_id')))\
.withColumn('start',when(col('strend') == min('strend').over(Window.partitionBy('group_id')),col('log_id')).otherwise(0))\
.withColumn('end',when(col('strend')   == max('strend').over(Window.partitionBy('group_id')),col('log_id')).otherwise(0))

start_df = logic_df.select('start').withColumn('rnk',dense_rank().over(Window.orderBy(col('start')))).filter(col('rnk') != 1)
end_df = logic_df.select('end').withColumn('rnk',dense_rank().over(Window.orderBy(col('end')))).filter(col('rnk') != 1)

start_df.join(end_df,start_df.rnk == end_df.rnk,'inner').select('start','end').show()


+-----+---+
|start|end|
+-----+---+
|    1|  3|
|    7|  8|
|   10| 10|
+-----+---+



24/08/21 16:04:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 16:04:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 16:04:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 16:04:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 16:04:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 16:04:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [302]:
logs_df.createOrReplaceTempView('logsdf')

spark.sql('''
  with main as (select log_id, sum(is_new_group) OVER(ORDER BY log_id) as group_id  from 
                    (select log_id,case when (log_id - prev_log_id) > 1 then 1 else 0  end is_new_group from 
                        (select log_id,lag(log_id) OVER(ORDER BY log_id) as prev_log_id from logsdf)) group by log_id,is_new_group),    
      start as (select distinct min(log_id) OVER(PARTITION BY group_id ) as start from main),
        end as (select distinct max(log_id) OVER(PARTITION BY group_id ) as end from main),
     end_id as (select end,row_number() OVER(ORDER BY end) as row_id from end), 
   start_id as (select start,row_number() OVER(ORDER BY start) as row_id from start)
                select start,end from start_id join end_id on start_id.row_id == end_id.row_id
         ''').show()

+-----+---+
|start|end|
+-----+---+
|    1|  3|
|    7|  8|
|   10| 10|
+-----+---+



24/08/21 19:58:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 19:58:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 19:58:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 19:58:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 19:58:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 19:58:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 39 The Most Frequently Ordered Products for Each Customer MP

In [303]:
customers_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/39_customers.csv'))

In [304]:
orders_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/39_orders.csv'))

In [305]:
products_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/39_products.csv'))

In [306]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [307]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)



In [308]:
products_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: integer (nullable = true)



In [333]:
window_spec = Window.partitionBy('customer_id','product_id')

orders_df.select('customer_id','product_id').withColumn('cnt',count(col('product_id')).over(window_spec))\
.withColumn('rnk',rank().over(Window.partitionBy('customer_id').orderBy(col('cnt').desc())).alias('rnk'))\
.filter(col('rnk') == 1).select('customer_id','product_id').distinct().orderBy('customer_id')\
.join(products_df,products_df.product_id == orders_df.product_id ,'inner')\
.select('customer_id',products_df.product_id,'product_name').orderBy('customer_id').show()

+-----------+----------+------------+
|customer_id|product_id|product_name|
+-----------+----------+------------+
|          1|         2|       mouse|
|          2|         2|       mouse|
|          2|         1|    keyboard|
|          2|         3|      screen|
|          3|         3|      screen|
|          4|         1|    keyboard|
+-----------+----------+------------+



In [340]:
orders_df.createOrReplaceTempView('ordersdf')
products_df.createOrReplaceTempView('productsdf')

spark.sql('''
   with main as (select customer_id,product_id,rank() OVER(PARTITION BY customer_id ORDER BY 
                     count(product_id) OVER(PARTITION BY customer_id,product_id) DESC) as rnk from ordersdf) 
                select distinct customer_id,main.product_id as product_id,product_name from main join 
                     productsdf on main.product_id = productsdf.product_id where rnk == 1 order by customer_id
         ''').show()

+-----------+----------+------------+
|customer_id|product_id|product_name|
+-----------+----------+------------+
|          1|         2|       mouse|
|          2|         2|       mouse|
|          2|         3|      screen|
|          2|         1|    keyboard|
|          3|         3|      screen|
|          4|         1|    keyboard|
+-----------+----------+------------+



#### 40 Biggest Window Between Visits MP

In [341]:
uservisits_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/40_uservisits.csv'))

In [342]:
uservisits_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- visit_date: string (nullable = true)



In [541]:
window_spec = Window.partitionBy('user_id').orderBy('visit_date')
uservisits_df.withColumn('diff',when( lead('visit_date',1).over(window_spec).isNull(),'2021-1-1')\
   .otherwise(lead('visit_date',1).over(window_spec)) )\
   .groupBy('user_id').agg(max(datediff('diff','visit_date')).alias('days')).orderBy('user_id').show()

+-------+----+
|user_id|days|
+-------+----+
|      1|  39|
|      2|  65|
|      3|  51|
+-------+----+



In [496]:
x = uservisits_df.select('user_id').distinct().collect()
lst = []
for i in range(0,len(x)):
    user_id = x[i][0]
    lst.append(
    uservisits_df.withColumn('rnk',rank().over(window_spec)).filter(col('user_id') == x[i][0] )\
   .withColumn('diff',when( lead('visit_date',1,0).over(Window.orderBy('visit_date')) == 0,'2021-1-1')\
   .otherwise(lead('visit_date',1,0).over(Window.orderBy('visit_date'))) )\
   .select(lit(user_id).alias('user_id'),max(expr("datediff(diff,visit_date)")).alias('days')).collect())

flattened_rows = [row for sublist in lst for row in sublist]

spark.createDataFrame(flattened_rows).orderBy('user_id').show()

24/08/21 23:18:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/21 23:18:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----+
|user_id|days|
+-------+----+
|      1|  39|
|      2|  65|
|      3|  51|
+-------+----+



24/08/21 23:18:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [520]:
spark.sql('''
        with main as (select user_id,visit_date,
                      CASE WHEN lead(visit_date,1,0) OVER(PARTITION BY user_id ORDER BY visit_date) == 0 THEN '2021-1-1'
                           ELSE lead(visit_date,1,0) OVER(PARTITION BY user_id ORDER BY visit_date) 
                           END lead_date 
                      FROM uservisitsdf)
                select user_id,max(datediff(lead_date,visit_date)) as days from main group by user_id order by user_id
         ''').show()

+-------+----+
|user_id|days|
+-------+----+
|      1|  39|
|      2|  65|
|      3|  51|
+-------+----+



#### 41 All People Report to the Given Manager MP

In [36]:
employees_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/41_employees.csv'))

In [37]:
employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- manager_id: integer (nullable = true)



In [101]:
lst = employees_df.select('manager_id','employee_id')\
.join(employees_df.select('employee_id','manager_id').withColumnRenamed('employee_id','manager_id'),on = 'manager_id')\
.collect()

In [137]:
spark.createDataFrame(lst).show()

+----------+-----------+----------+
|manager_id|employee_id|manager_id|
+----------+-----------+----------+
|         1|          1|         1|
|         3|          3|         3|
|         1|          2|         1|
|         2|          4|         1|
|         4|          7|         2|
|         3|          8|         3|
|         8|          9|         3|
|         1|         77|         1|
+----------+-----------+----------+



In [116]:
spark.createDataFrame(lst,["manager_id_1", "employee_id", "manager_id_2"])\
.filter( (col('manager_id_2') <= 2) & (col('employee_id') != 1) ).select('employee_id').show()

+-----------+
|employee_id|
+-----------+
|          2|
|          4|
|          7|
|         77|
+-----------+



In [149]:
employees_df.createOrReplaceTempView('employeesdf')

spark.sql('''
   with   three as (select one.employee_id,one.manager_id,two.employee_id as emp ,two.manager_id  from employeesdf as one 
               join employeesdf as two on one.employee_id == two.manager_id 
               where one.manager_id <= 2 and two.employee_id != 1 )
               select emp from three
               ''').show()

+---+
|emp|
+---+
|  2|
|  4|
|  7|
| 77|
+---+



#### 42 Page Recommendations MP

In [187]:
friendship_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/42_friendship.csv'))

liks_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/42_likes.csv'))

In [188]:
friendship_df.printSchema()

root
 |-- user1_id: integer (nullable = true)
 |-- user2_id: integer (nullable = true)



In [189]:
liks_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- page_id: integer (nullable = true)



#### 43 Students With Invalid Departments EP

In [217]:
departments_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/43_departments.csv'))

students_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/43_students.csv'))

In [218]:
departments_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [219]:
students_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- department_id: integer (nullable = true)



In [242]:
students_df.join(departments_df,students_df.department_id == departments_df.id, 'anti')\
 .select('id','name').orderBy('id').show()


+---+-------+
| id|   name|
+---+-------+
|  2|   John|
|  3|  Steve|
|  4|Jasmine|
|  7| Daiana|
+---+-------+



In [241]:
students_df.createOrReplaceTempView('studentsdf')
departments_df.createOrReplaceTempView('departmentsdf')

spark.sql('''
          select id,name from studentsdf anti join departmentsdf on 
               studentsdf.department_id == departmentsdf.id order by id
         ''').show()

+---+-------+
| id|   name|
+---+-------+
|  2|   John|
|  3|  Steve|
|  4|Jasmine|
|  7| Daiana|
+---+-------+



#### 44 Department Highest Salary M

In [244]:
employee_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/43_employee.csv'))

department_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/43_department.csv'))

In [245]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: integer (nullable = true)



In [246]:
department_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [260]:
window_spec = Window.partitionBy('dep_name').orderBy(col('salary').desc())
employee_df.join(department_df,employee_df.department == department_df.id,'inner')\
.select((employee_df.name).alias('emp_name'),(department_df.name).alias('dep_name'),'salary')\
.withColumn('mx',rank().over(window_spec)).filter(col('mx') == 1).select('dep_name','emp_name','salary').show()

+--------+--------+------+
|dep_name|emp_name|salary|
+--------+--------+------+
|   Sales|   Henry| 80000|
|      IT|     Jim| 90000|
|      IT|     Max| 90000|
+--------+--------+------+



In [296]:
employee_df.createOrReplaceTempView('employeedf')
department_df.createOrReplaceTempView('departmentdf')

spark.sql(''' select dep_name,emp_name,salary from 
              (select dep.name as dep_name,emp.name as emp_name,emp.salary,
                      rank() OVER(PARTITION BY department ORDER BY salary) as rnk from employeedf as emp
                        join departmentdf as dep on dep.id = emp.department) where rnk = 2
          ''').show()

+--------+--------+------+
|dep_name|emp_name|salary|
+--------+--------+------+
|      IT|     Jim| 90000|
|      IT|     Max| 90000|
|   Sales|   Henry| 80000|
+--------+--------+------+



#### 45 The Most Recent Orders for Each Product MP

In [385]:
customers_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../dataadvsql/45_customers.csv'))

orders_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/45_orders.csv'))

products_df = (spark.read
               .option('header',True)
               .option('inferSchema',True)
               .format('csv')
               .load('../../data/advsql/45_products.csv')
               .withColumnRenamed('product_id','productid'))

In [386]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [387]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)



In [388]:
products_df.printSchema()

root
 |-- productid: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: integer (nullable = true)



In [389]:
orders_df.show()

+--------+----------+-----------+----------+
|order_id|order_date|customer_id|product_id|
+--------+----------+-----------+----------+
|       1|2020-07-31|          1|         1|
|       2|2020-07-30|          2|         2|
|       3|2020-08-29|          3|         3|
|       4|2020-07-29|          4|         1|
|       5|2020-06-10|          1|         2|
|       6|2020-08-01|          2|         1|
|       7|2020-08-01|          3|         1|
|       8|2020-08-03|          1|         2|
|       9|2020-08-07|          2|         3|
|      10|2020-07-15|          1|         2|
+--------+----------+-----------+----------+



In [390]:
window_spec = Window.partitionBy('product_id').orderBy(col('order_date').desc())

orders_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1)\
.join(products_df,orders_df.product_id == products_df.productid,'inner' ) \
.select('product_name','product_id','order_id','order_date').show()

+------------+----------+--------+----------+
|product_name|product_id|order_id|order_date|
+------------+----------+--------+----------+
|    keyboard|         1|       6|2020-08-01|
|    keyboard|         1|       7|2020-08-01|
|      screen|         3|       3|2020-08-29|
|       mouse|         2|       8|2020-08-03|
+------------+----------+--------+----------+



In [392]:
orders_df.createOrReplaceTempView('ordersdf')
products_df.createOrReplaceTempView('productsdf')

spark.sql('''
          select product_name,product_id,order_id,order_date from 
          (select product_name,product_id,order_id,order_date,
               rank() OVER( PARTITION BY product_id ORDER BY order_date DESC ) as rnk from ordersdf
               join productsdf on productid = product_id) where rnk = 1
         ''').show()

+------------+----------+--------+----------+
|product_name|product_id|order_id|order_date|
+------------+----------+--------+----------+
|    keyboard|         1|       6|2020-08-01|
|    keyboard|         1|       7|2020-08-01|
|      screen|         3|       3|2020-08-29|
|       mouse|         2|       8|2020-08-03|
+------------+----------+--------+----------+

